# SVM model for toxic YouTube comments

En este notebook entrenamos un modelo de **SVM (LinearSVC)** para detectar
comentarios de odio/toxicidad en YouTube.

Objetivos:

- Cargar el dataset **preprocesado** (texto limpio/lematizado + etiquetas).
- Definir columna de texto y columna objetivo (`IsToxic` u otra).
- Vectorizar el texto con **TF-IDF** (unigramas y bigramas).
- Entrenar un modelo **LinearSVC** con `class_weight="balanced"`.
- Evaluar el modelo (accuracy, precision, recall, f1, ROC-AUC).
- Guardar el modelo entrenado como **`.pkl`** en `models/`.
- Guardar las métricas en un **`.json`** en `results/` con el formato acordado.


## 1. Imports y configuración

Importamos todas las librerías necesarias para:

- Carga de datos y manejo de rutas (`pandas`, `pathlib`).
- Modelado (`scikit-learn`).
- Guardado de modelo (`joblib`) y métricas (`json`).
- Medición de tiempo y timestamp (`datetime`).


In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import json
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Carga del dataset preprocesado

Cargamos el CSV **ya preprocesado** generado en el notebook de preprocessing
(`preprocessing_eda.ipynb` o similar).

- Ajusta la ruta `PREPROCESSED_PATH` según el nombre real de tu fichero.
- Este CSV debería contener:
  - Una columna de texto procesado (`text_processed` o similar).
  - La columna objetivo (`IsToxic` u otra).
  - Opcionalmente, features numéricos diseñados a mano (longitud, mayúsculas, etc.).


In [3]:
# Ajusta esta ruta al nombre del CSV que generaste en preprocessing
PREPROCESSED_PATH = Path("../../data/preprocessing_data/youtoxic_english_1000_clean.csv")

df = pd.read_csv(PREPROCESSED_PATH)

print("Shape del dataset preprocesado:", df.shape)
df.head()


Shape del dataset preprocesado: (997, 18)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsReligiousHate,text_basic,text_classic,text_len_classic,word_count_classic,uppercase_ratio,exclamation_count,hate_words_count
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,If only people would just take a step back and...,people would take step back make case wasnt an...,850,129,0.014121,0,2
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,Law enforcement is not trained to shoot to app...,law enforcement trained shoot apprehend traine...,90,13,0.036232,0,3
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,Dont you reckon them 'black lives matter' bann...,dont reckon black life matter banner held whit...,252,40,0.002375,0,1
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,There are a very large number of people who do...,large number people like police officer called...,339,49,0.015464,0,0
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,"The Arab dude is absolutely right, he should h...",arab dude absolutely right shot extra time sho...,138,23,0.020576,0,1
